In [1]:
from tika import parser
import os
import minecart
import warnings
import json
import pytesseract
from nltk.tokenize import  word_tokenize
import re
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

C:\Users\Melvin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Melvin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\Melvin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
#Globale Variablen definieren
directory = 'Betriebsvereinbarungen'

# Dictionary mit ID,Inhalt und ID, Dokumentenname
pdfs, lookupTable = dict(), dict()

#Dateien mit gescanntem Inhalt
badFiles = []

In [3]:
#Text und Metadaten der PDFs extrahieren 
index = 0
for filename in os.listdir(directory):
        lookupTable[index] = filename
        pdfs[filename] = parser.from_file(os.path.join(directoryG, filename))
        pdfs[filename]['ID'] = index
        index+=1

In [6]:
#Die Bilder der gescannten Dokumente extrahieren und OCR Software verwenden
for filename in badFiles:
    pdf = minecart.Document(open(os.path.join(directoryG, filename), 'rb'))
    #Durch alle Seiten iterieren
    text = ''
    for page in pdf.iter_pages():
        im = page.images[0].as_pil()  
        text+= pytesseract.image_to_string(im,lang='deu')
    pdfs[filename]['content'] = text  

In [7]:
# Text aufbereiten
data = dict()
for key, value in pdfs.items():
    temp = {'title': key,'content': value['content']}
    data[pdfs[key]['ID']] = temp

In [4]:
#Extarhierte Daten speichern
with open('Data/Data.json', 'w',encoding='utf-8') as fp:
    json.dump(data, fp,  indent=4,ensure_ascii=False)
    
with open('Data/FileIndex.json', 'w',encoding='utf-8') as fp:
    json.dump(lookupTable, fp,  indent=4,ensure_ascii=False)

In [2]:
# Vokabular für Rechtschreibkorrektur aufbauen
vocab = dict()

In [ ]:
# Methode zur Bereinigung der Daten
def cleanData(text):
    text = text.lower()
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub('[^a-zA-ZäöüÄÖÜß]', ' ', text)
    return text

In [ ]:
#Die Daten und Testanfragen laden
with open("Data/Data.json",encoding='utf-8') as file:
    data = json.load(file)

with open("Data/Queries.json",encoding='utf-8') as file:
    queries = json.load(file)

In [ ]:
# Vokabular anhand des Dokumentenkorpus mit Frequenz aufbauen
for key,value in data.items():
    text = cleanData(value['content'])
    text = word_tokenize(text.lower(), language='german')
    for token in text:
        if len(token) != 0:
            if token[0] in vocab:
                if token in vocab[token[0]]:
                    vocab[token[0]][token] += 1
                else:
                    vocab[token[0]][token] = 1
            else:
                newDict = {token : 1}
                vocab[token[0]] = newDict

In [ ]:
# Vokabular um Terme in Anfragen anreichern
for q in queries:
    text = cleanData(q[0])
    text = word_tokenize(text.lower(), language='german')
    for token in text:
        if len(token) != 0:
            if token[0] in vocab:
                if token in vocab[token[0]]:
                    vocab[token[0]][token] += 1
                else:
                    vocab[token[0]][token] = 1
            else:
                newDict = {token : 1}
                vocab[token[0]] = newDict     

In [ ]:
# Vokabular abspeichern
with open('Data/Vocab.json', 'w',encoding='utf-8') as fp:
    json.dump(vocab, fp,  indent=4, ensure_ascii=False)